FRED data: 
- VIX (volatility index): Measures market fear/uncertainty. Spikes during crises.
- 10Y treasury: 
- 2Y treasury:
- Yield curve: (10Y - 2Y) Predicts economic cycles. Inversion often precedes recessions
- BAA corporate Bond: Captures perceived credit risk in corporate bonds. Widens during financial stress.
- CPI: Indicates inflationary environment. Influences policy and real returns.
- Unemployment rate: Reflects slack in the labour market. Lags the business cycle but still valuable.
- Fed Funds Rate: Measures monetary policy stance. Sharp changes can trigger regime shifts.
- GDP growth: measures real economic activity (however it is in quarterly not monthly so may have to forward fill to each corresponding month i.e. fill each missing month with the most recent known GDP value)
- Credit spread (BAA - GS10) (calculated)
- YoY Inflation = 12-month % change in CPI 

API key: 043d7bab55a44c38c28c692cbfddae0e 

In [56]:
import pandas as pd
from datetime import datetime
from fredapi import Fred


In [57]:
f = Fred(api_key='043d7bab55a44c38c28c692cbfddae0e') # Instantiate FRED with API key 


In [58]:
# Define series of interest
series = {
    'VIX': 'VIXCLS',
    '10Y': 'GS10',
    '2Y': 'GS2',
    'Yield Curve (10Y - 2Y)': 'T10Y2Y',
    'BAA': 'BAA', # Corporate bond yield (credit risk)
    'CPI': 'CPIAUCSL',
    'UnemploymentRate': 'UNRATE',
    'FedFundsRate': 'FEDFUNDS',
    'GDP_YoY': 'A191RL1Q225SBEA'
}

In [59]:
start_date = '1963-07-01'
end_date = '2025-05-31'


In [60]:
macro_list = []
for name, code in series.items():
    data = f.get_series(code, observation_start=start_date, observation_end=end_date)
    df = data.to_frame(name)
    df['Date'] = df.index
    macro_list.append(df)


In [61]:
macro_df = pd.concat(macro_list, axis=1)
macro_df = macro_df.loc[:,~macro_df.columns.duplicated()]  # Remove duplicate 'date' columns
macro_df.reset_index(drop=True, inplace=True)


In [62]:
macro_df['CreditSpread'] = macro_df['BAA'] - macro_df['10Y']
macro_df['Inflation_YoY'] = macro_df['CPI'].pct_change(periods=12) * 100
macro_df['GDP_YoY'] = macro_df['GDP_YoY'].ffill()


C:\Users\User\AppData\Local\Temp\ipykernel_3400\244976231.py:2: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  macro_df['Inflation_YoY'] = macro_df['CPI'].pct_change(periods=12) * 100


In [63]:
final_columns = ['Date', 'VIX', '2Y', '10Y', 'YieldCurve', 'CreditSpread',
                 'FedFundsRate', 'Inflation_YoY', 'UnemploymentRate', 'GDP_YoY']
macro_df = macro_df[final_columns]

# 8. Export or preview
macro_df.to_csv('macro_dataset_cleaned.csv', index=False)


KeyError: "['YieldCurve'] not in index"

In [ ]:

print(macro_df.head())
